In [1]:
import json
from prettytable import PrettyTable

In [29]:
woz_path = "../../../data/woz/train.json"
with open(woz_path, "rb") as f:
    woz = json.load(f)

In [36]:
woz['dialogues'][0]

{'turns': [{'system_acts': [],
   'system_transcript': '',
   'num': {'system_acts': [[0, 16, 1]],
    'transcript': [0, 106, 48, 120, 301, 141, 9, 14, 33, 1]},
   'belief_state': [{'slots': [['food', 'eritrean']], 'act': 'inform'}],
   'turn_id': 0,
   'transcript': ['Are',
    'there',
    'any',
    'eritrean',
    'restaurants',
    'in',
    'town',
    '?'],
   'turn_label': [['food', 'eritrean']]},
  {'system_acts': [],
   'system_transcript': 'No, there are no eritrean restaurants in town. Would you like a different restaurant? ',
   'num': {'system_acts': [[0, 16, 1]],
    'transcript': [0, 79, 90, 94, 62, 33, 1]},
   'belief_state': [{'slots': [['food', 'chinese']], 'act': 'inform'}],
   'turn_id': 1,
   'transcript': ['How', 'about', 'Chinese', 'food', '?'],
   'turn_label': [['food', 'chinese']]},
  {'system_acts': [['request', 'area']],
   'system_transcript': 'There is a wide variety of Chinese restaurants, do you have an area preference or a price preference to narrow it

In [16]:
pred_path = "./test.pred.json"
ans_path = "../../../data/covid/ann/test.json"
ana_path = "../../../data/covid/ontology.json"

In [17]:
with open(pred_path, "rb") as f:
    pred = json.load(f)
with open(ans_path, "rb") as f:
    ans = json.load(f)
with open(ana_path, "rb") as f:
    ana = json.load(f)

In [18]:
for a in ans['dialogues']:
    if a['dialogue_index'] == 120:
        print(a['turns'][0]['turn_label'])
for a in pred['dialogues']:
    if a['dialogue_index'] == 120:
        print(a['turns'][0]['pred'])

In [19]:
a['turns'][0]["pred"]

[]

In [20]:
pred.keys()

dict_keys(['dialogues'])

In [21]:
len(pred['dialogues'])

60

In [22]:
len(ans['dialogues'])

60

In [23]:
slots = ana['slots']
slots

['fever',
 'cough',
 'phlegm',
 'tiredness',
 'breath_difficulty',
 'muscle_aches',
 'chills',
 'sore_throat',
 'runny_nose',
 'headache',
 'chest_pain',
 'nausea',
 'vomiting',
 'diarrhea',
 'unwell_duration',
 'age',
 'smoking',
 'heart_failure',
 'type_1_diabetes',
 'asthma',
 'chronic_lung_diseases',
 'high_blood_pressure',
 'recent_travel',
 'contact_with_infestor',
 'mass_gathering']

In [24]:
ans['dialogues'][0]["turns"][0]['turn_label']

[]

In [25]:
pred['dialogues'][0]["turns"][0]['pred']

[]

In [26]:
l1 = ['headache', 'yes']
l2 = ['headache', 'yyyes']
set(l1).union(set(l2))

{'headache', 'yes', 'yyyes'}

In [27]:
from collections import defaultdict
count = defaultdict(int)
for key in ana['slots']:
    count[key] = [0,0,0,0] # ans num, correct pred TP, wrong pred FN, not in pred FP
    
for i in range(len(pred['dialogues'])):
    p = pred['dialogues'][i]["turns"][0]['pred']
    p_d = {}
    for p_single in p:
        p_d[p_single[0]] = p_single[1]
    a = ans['dialogues'][i]["turns"][0]['turn_label']
    a_d = {}
    for a_single in a:
        a_d[a_single[0]] = a_single[1]
        
    l1 = a_d.keys()
    l2 = p_d.keys()
    
    keys = set(l1).union(set(l2)) # keys in p_d
    
    
    for aa in keys:
        try:
            count[aa][0] += 1
        except:
            print(aa, 'not in dict')
        if aa in a_d and aa not in p_d: # not in pred FP
            count[aa][2] += 1
        elif (aa in p_d and aa not in a_d) or p_d[aa] != a_d[aa]: # wrong pred FN
            count[aa][3] += 1
        elif p_d[aa] == a_d[aa]: # correct # TP
            count[aa][1] += 1
print(count)

defaultdict(<class 'int'>, {'fever': [6, 0, 6, 0], 'cough': [16, 9, 3, 4], 'phlegm': [4, 0, 4, 0], 'tiredness': [5, 1, 4, 0], 'breath_difficulty': [6, 1, 4, 1], 'muscle_aches': [7, 0, 7, 0], 'chills': [3, 0, 3, 0], 'sore_throat': [12, 7, 2, 3], 'runny_nose': [3, 2, 1, 0], 'headache': [7, 4, 2, 1], 'chest_pain': [2, 0, 1, 1], 'nausea': [4, 2, 2, 0], 'vomiting': [1, 1, 0, 0], 'diarrhea': [3, 2, 0, 1], 'unwell_duration': [3, 0, 2, 1], 'age': [4, 0, 4, 0], 'smoking': [2, 0, 1, 1], 'heart_failure': [2, 0, 2, 0], 'type_1_diabetes': [4, 0, 3, 1], 'asthma': [2, 0, 1, 1], 'chronic_lung_diseases': [5, 0, 5, 0], 'high_blood_pressure': [3, 0, 3, 0], 'recent_travel': [2, 0, 2, 0], 'contact_with_infestor': [5, 1, 4, 0], 'mass_gathering': [2, 0, 2, 0]})


In [28]:
metrics = defaultdict(int)
slots = ana['slots']
for s in slots:
    metrics[s] = []
    if count[s][1] + count[s][3]:
        metrics[s].append(count[s][1]/(count[s][1] + count[s][3])) # precision
    else:
        metrics[s].append(0.0)
        
    if count[s][1] + count[s][2]:
        metrics[s].append(count[s][1]/(count[s][1] + count[s][2])) # recall
    else:
        metrics[s].append(0.0)
        
    if metrics[s] != [0,0]:
        metrics[s].append(2*metrics[s][0]*metrics[s][1] / (metrics[s][0] + metrics[s][1]))
    else:
        metrics[s].append(0.0)
    print(metrics[s])

[0.0, 0.0, 0.0]
[0.6923076923076923, 0.75, 0.7199999999999999]
[0.0, 0.0, 0.0]
[1.0, 0.2, 0.33333333333333337]
[0.5, 0.2, 0.28571428571428575]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.7, 0.7777777777777778, 0.7368421052631577]
[1.0, 0.6666666666666666, 0.8]
[0.8, 0.6666666666666666, 0.7272727272727272]
[0.0, 0.0, 0.0]
[1.0, 0.5, 0.6666666666666666]
[1.0, 1.0, 1.0]
[0.6666666666666666, 1.0, 0.8]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
[1.0, 0.2, 0.33333333333333337]
[0.0, 0.0, 0.0]


In [15]:
for s in slots:
    print(s)
    table = PrettyTable()
    table.field_names = ['total number','correct predict','wrong predict','not in predict']
    table.add_row([count[s][0],str(count[s][1])[:5],str(count[s][2])[:5],str(count[s][3])[:5]])
    print(table,)
    
    table = PrettyTable()
    table.field_names = ['precision','recall','F1']
    table.add_row([str(metrics[s][0])[:5],str(metrics[s][1])[:5],str(metrics[s][2])[:5]])
    print(table,'\n\n')

fever
+--------------+-----------------+---------------+----------------+
| total number | correct predict | wrong predict | not in predict |
+--------------+-----------------+---------------+----------------+
|      6       |        0        |       6       |       0        |
+--------------+-----------------+---------------+----------------+
+-----------+--------+-----+
| precision | recall |  F1 |
+-----------+--------+-----+
|    0.0    |  0.0   | 0.0 |
+-----------+--------+-----+ 


cough
+--------------+-----------------+---------------+----------------+
| total number | correct predict | wrong predict | not in predict |
+--------------+-----------------+---------------+----------------+
|      16      |        9        |       3       |       4        |
+--------------+-----------------+---------------+----------------+
+-----------+--------+-------+
| precision | recall |   F1  |
+-----------+--------+-------+
|   0.692   |  0.75  | 0.719 |
+-----------+--------+-------+ 


ph